In [41]:
import sys
import glob
import numpy
from random import shuffle

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [43]:
def sz(a): # str/eval
    return {
        'text': a.text,
        'title': a.title,
        'url': a.url,
        'keywords': a.keywords,
        'tags': a.tags,
        'summary': a.summary,
        'date': int(a.publish_date.timestamp()) if a.publish_date != None else None,
        'raw' : a.html
    }

In [44]:
def objRead(f):
    return eval(open(f, 'r').read())

def objWrite(f, data):
    f = f.split('.')[0]
    open(f + '.lobj', 'w').write(str(data)) # light obj

In [45]:
for f in glob.iglob('**/*.lobj', recursive=True):
    print(f)

other.lobj
disasters/shootings/sandyhook.lobj
disasters/shootings/orlando.lobj
disasters/shootings/lasvegas.lobj
disasters/shootings/sanbernardino.lobj
disasters/earthquakes/sumatra.lobj
disasters/earthquakes/haiti.lobj
disasters/earthquakes/sichuan.lobj
disasters/earthquakes/kashmir.lobj
disasters/wildfires/thomas.lobj
disasters/wildfires/attica.lobj
disasters/wildfires/mountcarmel.lobj
disasters/wildfires/nevada.lobj
disasters/hurricanes/jeanne.lobj
disasters/hurricanes/stan.lobj
disasters/hurricanes/maria.lobj
disasters/hurricanes/katrina.lobj


In [46]:
data = [
    { 'articles': objRead('disasters/shootings/lasvegas.lobj'), 'label': 'lasvegas' },
    { 'articles': objRead('disasters/shootings/orlando.lobj'), 'label': 'orlando' },
    { 'articles': objRead('disasters/shootings/sandyhook.lobj'), 'label': 'sandyhook' },
    { 'articles': objRead('disasters/shootings/sanbernardino.lobj'), 'label': 'sanbernardino' },
    { 'articles': objRead('disasters/earthquakes/kashmir.lobj'), 'label': 'kashmir' },
    { 'articles': objRead('disasters/earthquakes/haiti.lobj'), 'label': 'haiti' },
    { 'articles': objRead('disasters/earthquakes/sumatra.lobj'), 'label': 'sumatra' },
    { 'articles': objRead('disasters/earthquakes/sichuan.lobj'), 'label': 'sichuan' },
#     { 'articles': objRead('disasters/wildfires/mountcarmel.lobj'), 'label': 'mountcarmel' }, # only 19 articles
    { 'articles': objRead('disasters/wildfires/attica.lobj'), 'label': 'attica' },
    { 'articles': objRead('disasters/wildfires/nevada.lobj'), 'label': 'nevada' },
    { 'articles': objRead('disasters/wildfires/thomas.lobj'), 'label': 'thomas' },
    { 'articles': objRead('disasters/hurricanes/maria.lobj'), 'label': 'maria' },
    { 'articles': objRead('disasters/hurricanes/katrina.lobj'), 'label': 'katrina' }
#     { 'articles': objRead('disasters/hurricanes/jeanne.lobj'), 'label': 'jeanne' }, # only 35 articles
#     { 'articles': objRead('disasters/hurricanes/stan.lobj'), 'label': 'stan'} # only 29 articles
]

labels = [x['label'] for x in data]

In [47]:
classifiers = [
    SGDClassifier(loss='hinge',
                  penalty='elasticnet',
                  alpha=1e-3,
                  max_iter=1000,
                  tol=None),
    MultinomialNB(),
    PassiveAggressiveClassifier(max_iter=200),
    KNeighborsClassifier(n_neighbors=len(data)),
    RandomForestClassifier(n_estimators=200,
                           max_depth=3),
    LogisticRegression(),
    LinearSVC()
]

In [48]:
for label in labels:    
    train_data, test_data = [], []
    train_label, test_label = numpy.array([]), numpy.array([])

    TRAIN_SIZE = 40

    for cat in data:
        l = int(cat['label'] != label)
        shuffle(cat['articles'])
        for x in cat['articles'][:TRAIN_SIZE]:
            if 'washingtonpost' in x['url']:
                continue
            train_data.append(x['text'])
            train_label = numpy.append(train_label, l)
        for x in cat['articles'][TRAIN_SIZE:]:
            if 'washingtonpost' in x['url']:
                continue
            test_data.append(x['text'])
            test_label = numpy.append(test_label, l)

    for c in classifiers:
        clf = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', c),
        ])
        clf.fit(train_data, train_label)
        pred_label = clf.predict(test_data)
        print(type(c).__name__, accuracy_score(test_label, pred_label))
        print(classification_report(test_label, pred_label, target_names = [label, 'others']))
        print(confusion_matrix(test_label, pred_label))

SGDClassifier 0.9863013698630136
             precision    recall  f1-score   support

   lasvegas       1.00      0.83      0.91        54
     others       0.99      1.00      0.99       603

avg / total       0.99      0.99      0.99       657

[[ 45   9]
 [  0 603]]
MultinomialNB 0.9178082191780822
             precision    recall  f1-score   support

   lasvegas       0.00      0.00      0.00        54
     others       0.92      1.00      0.96       603

avg / total       0.84      0.92      0.88       657

[[  0  54]
 [  0 603]]


/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


PassiveAggressiveClassifier 0.9832572298325722
             precision    recall  f1-score   support

   lasvegas       0.98      0.81      0.89        54
     others       0.98      1.00      0.99       603

avg / total       0.98      0.98      0.98       657

[[ 44  10]
 [  1 602]]
KNeighborsClassifier 0.969558599695586
             precision    recall  f1-score   support

   lasvegas       1.00      0.63      0.77        54
     others       0.97      1.00      0.98       603

avg / total       0.97      0.97      0.97       657

[[ 34  20]
 [  0 603]]
RandomForestClassifier 0.9223744292237442
             precision    recall  f1-score   support

   lasvegas       1.00      0.06      0.11        54
     others       0.92      1.00      0.96       603

avg / total       0.93      0.92      0.89       657

[[  3  51]
 [  0 603]]
LogisticRegression 0.9330289193302892
             precision    recall  f1-score   support

   lasvegas       1.00      0.19      0.31        54
     others  

/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


PassiveAggressiveClassifier 0.9847328244274809
             precision    recall  f1-score   support

    orlando       1.00      0.81      0.90        54
     others       0.98      1.00      0.99       601

avg / total       0.98      0.98      0.98       655

[[ 44  10]
 [  0 601]]
KNeighborsClassifier 0.9389312977099237
             precision    recall  f1-score   support

    orlando       1.00      0.26      0.41        54
     others       0.94      1.00      0.97       601

avg / total       0.94      0.94      0.92       655

[[ 14  40]
 [  0 601]]
RandomForestClassifier 0.9206106870229007
             precision    recall  f1-score   support

    orlando       1.00      0.04      0.07        54
     others       0.92      1.00      0.96       601

avg / total       0.93      0.92      0.89       655

[[  2  52]
 [  0 601]]
LogisticRegression 0.9404580152671755
             precision    recall  f1-score   support

    orlando       1.00      0.28      0.43        54
     others 

...

PassiveAggressiveClassifier 0.981679389312977
             precision    recall  f1-score   support

    katrina       0.98      0.80      0.88        54
     others       0.98      1.00      0.99       601

avg / total       0.98      0.98      0.98       655

[[ 43  11]
 [  1 600]]
KNeighborsClassifier 0.932824427480916
             precision    recall  f1-score   support

    katrina       1.00      0.19      0.31        54
     others       0.93      1.00      0.96       601

avg / total       0.94      0.93      0.91       655

[[ 10  44]
 [  0 601]]
RandomForestClassifier 0.917557251908397
             precision    recall  f1-score   support

    katrina       0.00      0.00      0.00        54
     others       0.92      1.00      0.96       601

avg / total       0.84      0.92      0.88       655

[[  0  54]
 [  0 601]]


/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LogisticRegression 0.917557251908397
             precision    recall  f1-score   support

    katrina       0.00      0.00      0.00        54
     others       0.92      1.00      0.96       601

avg / total       0.84      0.92      0.88       655

[[  0  54]
 [  0 601]]


/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LinearSVC 0.966412213740458
             precision    recall  f1-score   support

    katrina       1.00      0.59      0.74        54
     others       0.96      1.00      0.98       601

avg / total       0.97      0.97      0.96       655

[[ 32  22]
 [  0 601]]
